In [18]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Add,Concatenate,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed

from keras.callbacks import ModelCheckpoint

from keras.engine.topology import Layer, InputSpec
from keras import initializers
from sklearn.model_selection import train_test_split

#參數
MAX_SENT_LENGTH = 8
MAX_SENTS = 64
EMBEDDING_DIM = 300


## label data

print (keras.__version__)

column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)

labels = df['type'].values
labels = np.array(labels)
print(labels)
labels = np_utils.to_categorical(labels)
print(labels.shape)
tokenlizeword = np.load('./tokenlizeword0221.npy',allow_pickle=True)
u_tokenlizeword = np.load('./tokenlizeword0226_udn_for_mct.npy',allow_pickle=True)
#串起來
tokenlizeword=np.concatenate((tokenlizeword, u_tokenlizeword), axis=0)

wmodel = Word2Vec(tokenlizeword, size=300, window=5, min_count=0)
# wmodel=Word2Vec.load('./word2vec.model')
# wmodel.most_similar('我')

2.3.1
[0 0 2 ... 3 6 2]
(35546, 7)


In [19]:
#x_train=文章數*最大句數*最大字數*300維
x_train =[] #label data
##文章數
for k in range(tokenlizeword.shape[0]):
    embedding_matrix=np.zeros((MAX_SENTS,MAX_SENT_LENGTH,EMBEDDING_DIM))
    word_count=0 ##計算目前位置
    ##句子數
    for i in range(MAX_SENTS):
        if word_count >= len(tokenlizeword[k]):
            break
        embedding_vector=np.zeros((MAX_SENT_LENGTH,EMBEDDING_DIM))
        ##最大字數
        for j in range(MAX_SENT_LENGTH):
            if tokenlizeword[k][word_count] == '，' or tokenlizeword[k][word_count] == '。' or tokenlizeword[k][word_count] == '！' or tokenlizeword[k][word_count] == '？':
                word_count=word_count+1
                break
            else:
                embedding_vector[j]=wmodel[tokenlizeword[k][word_count]]
                word_count=word_count+1
                if word_count >= len(tokenlizeword[k]):
                    break
        embedding_matrix[i]=embedding_vector
    x_train.append(embedding_matrix)

print(len(x_train))
print("embedding done")
            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


46237
embedding done


In [21]:
x_train = np.array(x_train,dtype = 'float32')


dev_x = x_train[:5000]
dev_y = labels[:5000]

test_x = x_train[5000:10000]
test_y = labels[5000:10000]
#到35546為止為label data
train_x = x_train[10000:35546]

train_y = labels[10000:]
# !ls
print(dev_x.shape)
print(test_x.shape)
print(train_x.shape)

print("60%")

(5000, 64, 8, 300)
(5000, 64, 8, 300)
(25546, 64, 8, 300)
60%


In [ ]:
print(type(train_y[0]))

In [22]:
#注意層
class Attention(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = 50
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], 1)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(Attention, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)

        if mask is not None:
            ait *= K.cast(mask, K.floatx())
            
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [63]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
l_lstm = Bidirectional(GRU(100, return_sequences=True))(sentence_input)
l_dense = TimeDistributed(Dense(100))(l_lstm)
l_att = Attention()(l_dense)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(100))(l_lstm_sent)
l_att_sent = Attention()(l_dense_sent)
preds = Dense(7, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])
filepath = "./HAN.h5"
 
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max', period=1)
callbacks_list = [checkpoint]

model.summary()
print("model fitting - Hierachical attention network")
model.fit(train_x, train_y, validation_data=(dev_x,dev_y),epochs=3, batch_size=32,callbacks = callbacks_list)

tracking <tf.Variable 'attention_25/Variable:0' shape=(100, 1) dtype=float32> W
tracking <tf.Variable 'attention_25/Variable_1:0' shape=(50,) dtype=float32> b
tracking <tf.Variable 'attention_25/Variable_2:0' shape=(50, 1) dtype=float32> u
tracking <tf.Variable 'attention_26/Variable:0' shape=(100, 1) dtype=float32> W
tracking <tf.Variable 'attention_26/Variable_1:0' shape=(50,) dtype=float32> b
tracking <tf.Variable 'attention_26/Variable_2:0' shape=(50, 1) dtype=float32> u
Model: "model_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 64, 8, 300)        0         
_________________________________________________________________
time_distributed_39 (TimeDis (None, 64, 100)           260900    
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 64, 200)           120600    
________________________________________

In [10]:

sentence_input = Input(shape=(MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
l_lstm = Bidirectional(GRU(300, return_sequences=True))(sentence_input)
l_att = AttLayer(300)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
# l_drop1 = Dropout(0.3)(review_encoder)
# l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
dropout = Dropout(0.2)(review_encoder)
filter_sizes = [3,4,5]
convs = []
for filter_size in filter_sizes:
    conv = Conv1D(filters=64, kernel_size=filter_size, padding='same', activation='relu')(dropout)
    pool = MaxPooling1D(filter_size)(conv)
    convs.append(pool)
        
concatenate = Concatenate(axis=1)(convs)
drop_out = Dropout(0.1)(concatenate)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(drop_out)
preds = Dense(7, activation='softmax')(l_lstm_sent)
l_att_sent = AttLayer(100)(preds)

model = Model(review_input, l_att_sent)

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])


filepath = "best_HAHNN.h5"
 
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max', period=1)
callbacks_list = [checkpoint]


model.summary()
print("model fitting - Hierachical attention network")
model.fit(test_x, test_y, validation_data=(dev_x,dev_y),epochs=3, batch_size=4,callbacks=callbacks_list)

tracking <tf.Variable 'att_layer_4/W:0' shape=(200, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_4/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_4/u:0' shape=(100, 1) dtype=float32> u
tracking <tf.Variable 'att_layer_5/W:0' shape=(7, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_5/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_5/u:0' shape=(100, 1) dtype=float32> u
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 8, 300)   0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 64, 200)      260800      input_6[0][0]                    
__________________________________________________________

In [61]:
def test_acc(test_x,test_y,model):
    test_y = np.array(test_y,dtype ='int64')
    total = len(test_x)
    logits=model.predict(test_x,batch_size=50)
    pred=np.argmax(logits, axis=1)
    correct=0 #對的有幾個
    label_list = ['政治','生活','國際','體育','娛樂','社會','財經']
    total_labels =[0,0,0,0,0,0,0]
    correct_labels = [0,0,0,0,0,0,0]
    total_labels = np.array(total_labels)
    correct_labels = np.array(correct_labels)
    for i in range(len(test_x)):
        total_labels += test_y[i]
        if test_y[i][pred[i]]==1.0:
            correct = correct+1
            correct_labels[pred[i]] += 1
    for i in range(len(total_labels)):
            print(label_list[i],':',correct_labels[i],'/',total_labels[i], 'Acc :', (correct_labels[i]/total_labels[i]))
    print('Total：',correct/total)

In [70]:
## 有bug
def test_acc_two_ans(test_x,test_y,model):
    test_y = np.array(test_y,dtype ='int64')
    total = len(test_x)
    logits=model.predict(test_x,batch_size=50)
    pred=np.argmax(logits, axis=1)
    for i in range(len(logits)):
        logits[i][pred[i]] = 0.0
    pred_sec = np.argmax(logits,axis=1)
    correct=0 #對的有幾個
    label_list = ['政治','生活','國際','體育','娛樂','社會','財經']
    total_labels =[0,0,0,0,0,0,0]
    correct_labels = [0,0,0,0,0,0,0]
    total_labels = np.array(total_labels)
    correct_labels = np.array(correct_labels)
    for i in range(len(test_x)):
        total_labels += test_y[i]
        if test_y[i][pred[i]]==1.0 or test_y[i][pred_sec[i]]==1.0:
            correct = correct+1
            correct_labels+= test_y[i]
    for i in range(len(total_labels)):
            print(label_list[i],':',correct_labels[i],'/',total_labels[i], 'Acc :', (correct_labels[i]/total_labels[i]))
    print('Total：',correct/total)

In [71]:
test_acc_two_ans(test_x,test_y,model)
test_acc(test_x,test_y,model)

政治 : 914 / 1104 Acc : 0.8278985507246377
生活 : 1072 / 1107 Acc : 0.9683830171635049
國際 : 399 / 424 Acc : 0.9410377358490566
體育 : 358 / 359 Acc : 0.9972144846796658
娛樂 : 639 / 653 Acc : 0.9785604900459418
社會 : 379 / 453 Acc : 0.8366445916114791
財經 : 869 / 900 Acc : 0.9655555555555555
Total： 0.926
政治 : 793 / 1104 Acc : 0.7182971014492754
生活 : 929 / 1107 Acc : 0.8392050587172538
國際 : 314 / 424 Acc : 0.7405660377358491
體育 : 350 / 359 Acc : 0.9749303621169917
娛樂 : 611 / 653 Acc : 0.9356814701378254
社會 : 307 / 453 Acc : 0.6777041942604857
財經 : 800 / 900 Acc : 0.8888888888888888
Total： 0.8208


In [64]:

logits=model.predict(test_x,batch_size=50)
pred=np.argmax(logits, axis=1)

In [65]:
#驗證還沒加入unlabel data的表現
correct=0 #對的有幾個
for i in range(5000):
    if test_y[i][pred[i]]==1.0:
        correct = correct+1
print(correct)
acc=correct/5000
print(acc)

4104
0.8208


In [16]:
##預測
u_train = x_train[35546:]
u_pred = model.predict(u_train,batch_size=50)
ary =[] ##準備刪除的所有資料位置
for i in range(u_train.shape[0]):
    ans = u_pred[i][0]
    for j in range(7):
        if u_pred[i][j]>ans:
            ans=u_pred[i][j]
    if ans<acc: ##比模型準確率高
        ary.append(i)


In [17]:
print(u_pred[3])
print(u_pred.shape)
u_pred=np.delete(u_pred,ary,axis=0) ##刪除所有資料不夠好的位置
print(u_pred[3])
print(u_pred.shape)

[2.3992579e-01 2.1731112e-02 8.2282134e-04 7.1054637e-01 4.6229051e-04
 2.6327234e-02 1.8443231e-04]
(10691, 7)
[8.8439238e-01 5.6384900e-04 5.2494352e-06 9.5021096e-06 1.8242728e-06
 1.1501102e-01 1.6204893e-05]
(6374, 7)


In [20]:
ary_label = []##刪除x_train用ㄉ

for i in range(len(ary)):
    ary_label.append(ary[i]+35546)
x_train = np.delete(x_train,ary_label,axis=0)
print(x_train.shape)
u_pred = np.argmax(u_pred, axis=1)
##轉one-hot
u_pred = np.array(u_pred)
u_pred = np_utils.to_categorical(u_pred)
print(u_pred.shape)
print(u_pred[7])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """


MemoryError: Unable to allocate 44.9 GiB for an array with shape (39218, 64, 8, 300) and data type float64

In [ ]:
u_pred = np.argmax(u_pred, axis=1)
##轉one-hot
u_pred = np.array(u_pred)
u_pred = np_utils.to_categorical(u_pred)
print(u_pred.shape)
print(u_pred[7])
train_y=labels[10000:]

In [ ]:
#串起來
train_y=np.concatenate((train_y,u_pred),axis=0)
print(train_y.shape)

In [ ]:
#再train一個model
sentence_input = Input(shape=(MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
l_lstm = Bidirectional(GRU(100, return_sequences=True))(sentence_input)
dropout = Dropout(0.2)(l_lstm)
filter_sizes = [3,4,5]
convs = []
for filter_size in filter_sizes:
    conv = Conv1D(filters=64, kernel_size=filter_size, padding='same', activation='relu')(dropout)
    pool = MaxPooling1D(filter_size)(conv)
    convs.append(pool)
        
concatenate = Concatenate(axis=1)(convs)
drop_out = Dropout(0.1)(concatenate)
l_att = AttLayer(100)(drop_out)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH,EMBEDDING_DIM), dtype='float32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
# l_drop1 = Dropout(0.3)(review_encoder)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent)
preds = Dense(7, activation='softmax')(l_att_sent)
u_model = Model(review_input, preds)

u_model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])


filepath = "best_HAN_with_unlabel.h5"
 
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max', period=1)
callbacks_list = [checkpoint]

train_x=x_train[10000:]
print(train_x.shape)
print(train_y.shape)
u_model.summary()
print("model fitting - Hierachical attention network")
u_model.fit(train_x, train_y, validation_data=(dev_x,dev_y),epochs=3, batch_size=4,callbacks=callbacks_list)

In [15]:
pred=u_model.predict(test_x,batch_size=50)
pred=np.argmax(pred, axis=1)
print(pred.shape)


(5000,)


In [16]:
#加入unlabel data後的表現
correct=0 #對的有幾個
for i in range(5000):
    if test_y[i][pred[i]]==1.0:
        correct = correct+1
print(correct)
acc=correct/5000
print(acc)

4176
0.8352
